<a href="https://colab.research.google.com/github/lsteffenel/ED-SNI-IntroDL/blob/main/02_Introduction_a_Keras_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lsteffenel/ED-SNI-IntroDL/blob/main/02_Introduction_a_Keras_MNIST.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

## Construire un modèle de réseaux de neurones avec Keras

Keras est une bibliothèque "haut niveau" utilisée pour simplifier la description de modèles de réseaux de neurones sur Tensorflow (bibliothèque IA de Google). L'avantage surtout est de pouvoir utiliser des GPU pour accélérer le calcul.

Le travail avec Keras suit un cheminement similaires à celui avec Scikit-Learn, mais il y a quelques différences à retenir.

In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import math
import numpy as np
print (tf.__version__)


Dans le paragraphe suivant vous avez certainement eu un message d'erreur indiquant que vous n'avez pas des GPU. Dans ce cas, Keras utilisera la CPU de la machine.

## Chargement de données

Tout comme Scikit-Learn, Keras a aussi un ensemble de datasets prêt à utilisation pour des exemples. Dans le cas suivant, nous allons charger le dataset MNIST (écriture à la main) et le séparer en deux groupes : Train et Test. Les données de validation (vérification pendant l'entraînement) seront séparés du groupe Train plus tard.

In [ ]:
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Dans ce dataset, nous avons 70000 images. 60000 images sont utilisées pour l'entraînement, et 10000 pour l'ensemble de `test`. On peut visualiser les "dimensions" de chacun des ensembles :
- les ensembles "x" contiennent les données. On trouve respectivement 60000 et 10000 matrices 28x28.
- les ensembles "y" contiennent juste les résultats attendus (appelés "étiquettes" ou "label"). On trouve également 60000 et 10000 lignes. Comme les résultats se trouvent sur une "colonne", on n'a pas besoin d'indiquer la dimension "1" pour celle-ci.


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

On peut également voir le contenu des ensembles. Ici, on affiche `y_train` (l'affichage est tronquée).

In [ ]:
y_train

Comme on vient de voir, les données de MNIST se présentent sous la forme d'images 28x28 pixels. Ceux-ci peuvent avoir des valeurs de 0 à 255, correspondant à 256 tons de gris. Les labels (`y_train`, par exemple) correspondent aux caractères représentés : les chiffres 0 à 9.

Le paragraphe suivant définit une fonction permettant de visualiser ce dataset.

In [ ]:
import random

def plot_images(x,y=None,y_pred=None):
  nrows=3
  ncols=4
  draw_labels = (y is not None)
  draw_pred   = (y_pred is not None)

  fig, axs = plt.subplots(nrows, ncols, layout=None)
  indices = random.sample(range(0,1000),nrows*ncols)
  i = 0
  for ax in axs.flat:
    ax.imshow(x[indices[i]])
    if not draw_labels and not draw_pred:
      ax.set_xlabel(indices[i], fontsize=12)
    if draw_labels and not draw_pred:
      ax.set_xlabel(y[indices[i]], fontsize=12)
    if draw_labels and draw_pred:
      pred = str(y_pred[indices[i]])+' ('+str(y[indices[i]])+')'
      color = 'red' if y_pred[indices[i]] != y[indices[i]] else 'black'
      ax.set_xlabel(pred, fontsize=12, color=color)
    i+=1
  plt.subplots_adjust(hspace=0.3)
  plt.tight_layout()
  plt.show()

Remarquez le "label" correspondant sous chaque image.

In [ ]:
plot_images(x_train,y_train,y_train)

Les paragraphes suivants font plusieurs opérations afin de préparer les données :

**1 - Reformater les données**

Dans un réseau de neurones dense (DNN), chaque neurone reçoit l'ensemble des données. Pour simplifier cela, les images 28x28 seront "applaties" en un seul array unidimensionnel de 784 valeurs

In [ ]:
x_train = x_train.reshape(60000, 784) #  28*28
x_test = x_test.reshape(10000, 784)

**2 - Transformation et normalisation des données**

Les valeurs de base son des entiers entre 0 et 255 pour représenter les 256 tons de gris. La majorité des algorithmes utilisent des valeurs réels, de préférence dans la fourchette 0 à 1 ou -1 à 1.

Les paragraphes suivantes modifient le type des données (`float32`) puis font une normalisation simple (diviser la valeur par 255). Bien sûr, d'autres méthodes de normalisation plus élaborées sont possibles, mais ça suffit pour l'instant.

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
x_train /= 255
x_test /= 255

**3 - Transformer les données catégoriques**
Lorsqu'on a des données catégoriques (texte ou numéros), il faut les transformer afin d'éviter des mauvaises compréhensions de la part de l'algorithme (par exemple, supposer que une classe 2 vient toujours après une classe 1). Dans notre cas, nous allons transformer les classes 0 à 9 en représentations numériques (similaire à HotOneEncoder de Sklearn), afin de rendre indépendantes ces classes.

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
y_train

## La Création d'un modèle

Keras a plusieurs modes permettant la création de modèles de réseaux de neurones. Dans ce cas, nous allons utiliser l'API `Sequential` qui permet de décrire couche par couche du réseau et les empiler (grâce à `add()`).

Nous allons faire un modèle simple avec des réseaux denses (totalement connectés). La première couche définit la taille de l'entrée (les 784 valeurs reçus du dataset), les autres utilisent par défaut la taille de la sortie de la couche précédente. Egalement, nous indiquons que chaque couche comptera avec 10 neurones.

Finalemen, remarquez qu'on utilise deux types de fonction d'activation, sigmoid et softmax.
Pour simplifier la description, sigmoid donne une probabilité entre 0 et 1, alors que Softmax affiche "1" sur la sortie avec la plus grande probabilité et "0" sur les autres. C'est pour cela qu'on utilise Softmax à la sortie, ça permet d'avoir un résultat plutôt qu'une liste de probabilités.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# Declaration du modèle en Tensorflow2.0
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(20, activation='sigmoid', input_dim =(784)))
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))


# résumé du modèle
model.summary()



## Entraînement du modèle

Une fois défini le modèle, il faut l'entraîner avec les données.
Le paragraphe suivant définit les hyperparamètres du modèle, dont le `batch_size`(taille des sous-ensembles utilisés dans la descente de gradient), le nombre d'epochs (parcours de l'ensemble de données d'entraînement).

L'appel à compile indique aussi qu'on utilise le modèle de descente de gradient SGD (il y a plusieurs), que la métrique utilisée est l'accuracy (métrique qui correspond à (TP+TN)/(TP+TN+FP+FN)), et que la fonction de perte est la `categorical_crossentropy`, une fonction qui compare les probabilités pour des labels catégoriques.

In [ ]:
batch_size = 100
#num_classes = 10
epochs= 50

model.compile(loss='categorical_crossentropy',  optimizer='SGD',  metrics=['accuracy'])


Finalement, on lance l'entraînement. Remarquez aussi qu'on n'a pas crée des données Validation avant, on le fera ici en réservant 10% des données de Train.

Comme le dataset est simple, on peut faire 50 epoch même sans un GPU.

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,verbose=1 )

#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
# Je vous invite à lire la documentation : https://keras.io/models/sequential/

Les paragraphes suivants nous permettent de voir comment le modèle améliore sa performance au fil des epochs

In [ ]:
def plot_history(history, figsize=(8,6),
                 plot={"Accuracy":['accuracy','val_accuracy'], 'Loss':['loss', 'val_loss']}):
    """
    Show history
    args:
        history: history
        figsize: fig size
        plot: list of data to plot : {<title>:[<metrics>,...], ...}
    """
    fig_id=0
    for title,curves in plot.items():
        plt.figure(figsize=figsize)
        plt.title(title)
        plt.ylabel(title)
        plt.xlabel('Epoch')
        for c in curves:
            plt.plot(history.history[c])
        plt.legend(curves, loc='upper left')

        plt.show()

In [ ]:
plot_history(history, figsize=(6,4))

Enfin, on peut estimer la performance du modèle avec les données Test.

Comparez ces valeur avec ceux de l'entraînement (`val_loss` et `val_accuracy`
 ci-dessus).

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test,verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Ces résultats montrent que le modèle se porte un peu moins bien avec de nouvelles données, mais ça reste intéressant.

## Exercice :
On a obtenu avec ce modèle basique, un taux d'accuracy supérieur à 70%.
- Essayer d'améliorer la performence du modèle, en modifiant les fonctions d'activation, ou/et en n ajoutant le nombre de neurones et des couches intermédiaires.



In [ ]:
x_test = x_test.reshape(10000, 28,28)
# d'abord, on utilise le modèle pour faire une prévision sur l'ensemble de test
# Ça retourne une liste avec 10 colonnes (une par sortie possible).
y_sigmoid = model.predict(x_test.reshape(10000, 784))

# avec la fonction argmax, on ne garde que l'index de la colonne avec la plus grande valeur
y_pred    = np.argmax(y_sigmoid, axis=-1)
y_test_labels = np.argmax(y_test, axis=-1)

In [ ]:
y_pred

In [ ]:
# maintenant, on affiche quelques éléments, avec la valeur prédite et la valeur attendue (entre parenthèses)
plot_images(x_test,y_test_labels,y_pred)